The submissions should contain the predicted probability of click for each ad impression in the test set 

### Data fields
- id: ad identifier
- click: 0/1 for non-click/click
- hour: format is YYMMDDHH, so 14091123 means 23:00 on Sept. 11, 2014 UTC.
- C1 -- anonymized categorical variable
- banner_pos
- site_id
- site_domain
- site_category
- app_id
- app_domain
- app_category
- device_id
- device_ip
- device_model
- device_type
- device_conn_type
- C14-C21 -- anonymized categorical variables

In [1]:
import numpy as np
import pandas as pd
import time
import pandas_profiling

In [2]:
%%time
# start = time.time()
train_ori = pd.read_csv('./train.csv')
# mid = time.time()
# print(mid-start)
test_ori = pd.read_csv('./test.csv')
# end = time.time()
# print(end-mid)

Wall time: 7min 17s


In [3]:
len(train)

40428967

In [3]:
train.head()

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,...,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,1.000009e+18,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,2,15706,320,50,1722,0,35,-1,79
1,1.000017e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15704,320,50,1722,0,35,100084,79
2,1.000037e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15704,320,50,1722,0,35,100084,79
3,1.000064e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15706,320,50,1722,0,35,100084,79
4,1.000068e+19,0,14102100,1005,1,fe8cc448,9166c161,0569f928,ecad2386,7801e8d9,...,1,0,18993,320,50,2161,0,35,-1,157


In [4]:
train.tail()

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,...,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
40428962,9.998753e+18,1,14103023,1005,1,e151e245,7e091613,f028772b,ecad2386,7801e8d9,...,1,0,17262,320,50,1872,3,39,100173,23
40428963,9.999038e+18,0,14103023,1005,0,85f751fd,c4e18dd6,50e219e0,9c13b419,2347f47a,...,1,2,23160,320,50,2667,0,47,-1,221
40428964,9.999585e+18,0,14103023,1005,1,f61eaaae,6b59f079,f028772b,ecad2386,7801e8d9,...,1,0,20969,320,50,2372,0,813,-1,46
40428965,9.999636e+18,1,14103023,1005,0,85f751fd,c4e18dd6,50e219e0,3c4b944d,2347f47a,...,1,0,16859,320,50,1887,3,39,100194,23
40428966,9.999747e+18,0,14103023,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,22257,320,50,2545,0,431,100084,221


In [4]:
train.describe()

,id,click,hour,C1,banner_pos,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
count,4.042897e+07,4.042897e+07,4.042897e+07,4.042897e+07,4.042897e+07,4.042897e+07,4.042897e+07,4.042897e+07,4.042897e+07,4.042897e+07,4.042897e+07,4.042897e+07,4.042897e+07,4.042897e+07,4.042897e+07
mean,9.223017e+18,1.698056e-01,1.410256e+07,1.004968e+03,2.880146e-01,1.015305e+00,3.313150e-01,1.884181e+04,3.188831e+02,6.010201e+01,2.112601e+03,1.432499e+00,2.271444e+02,5.321685e+04,8.338229e+01
std,5.325443e+18,3.754620e-01,2.966837e+02,1.094586e+00,5.063820e-01,5.274336e-01,8.547935e-01,4.959457e+03,2.127250e+01,4.729538e+01,6.094124e+02,1.326227e+00,3.510221e+02,4.995682e+04,7.028996e+01
min,5.211594e+11,0.000000e+00,1.410210e+07,1.001000e+03,0.000000e+00,0.000000e+00,0.000000e+00,3.750000e+02,1.200000e+02,2.000000e+01,1.120000e+02,0.000000e+00,3.300000e+01,-1.000000e+00,1.000000e+00
25%,4.611181e+18,0.000000e+00,1.410230e+07,1.005000e+03,0.000000e+00,1.000000e+00,0.000000e+00,1.692000e+04,3.200000e+02,5.000000e+01,1.863000e+03,0.000000e+00,3.500000e+01,-1.000000e+00,2.300000e+01
50%,9.223224e+18,0.000000e+00,1.410260e+07,1.005000e+03,0.000000e+00,1.000000e+00,0.000000e+00,2.034600e+04,3.200000e+02,5.000000e+01,2.323000e+03,2.000000e+00,3.900000e+01,1.000480e+05,6.100000e+01
75%,1.383561e+19,0.000000e+00,1.410281e+07,1.005000e+03,1.000000e+00,1.000000e+00,0.000000e+00,2.189400e+04,3.200000e+02,5.000000e+01,2.526000e+03,3.000000e+00,1.710000e+02,1.000930e+05,1.010000e+02
max,1.844674e+19,1.000000e+00,1.410302e+07,1.012000e+03,7.000000e+00,5.000000e+00,5.000000e+00,2.405200e+04,1.024000e+03,1.024000e+03,2.758000e+03,3.000000e+00,1.959000e+03,1.002480e+05,2.550000e+02


In [12]:
dtrain = train.iloc[0:100000]

In [13]:
dtrain

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,...,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,1.000009e+18,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,2,15706,320,50,1722,0,35,-1,79
1,1.000017e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15704,320,50,1722,0,35,100084,79
2,1.000037e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15704,320,50,1722,0,35,100084,79
3,1.000064e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15706,320,50,1722,0,35,100084,79
4,1.000068e+19,0,14102100,1005,1,fe8cc448,9166c161,0569f928,ecad2386,7801e8d9,...,1,0,18993,320,50,2161,0,35,-1,157
5,1.000072e+19,0,14102100,1005,0,d6137915,bb1ef334,f028772b,ecad2386,7801e8d9,...,1,0,16920,320,50,1899,0,431,100077,117
6,1.000072e+19,0,14102100,1005,0,8fda644b,25d4cfcd,f028772b,ecad2386,7801e8d9,...,1,0,20362,320,50,2333,0,39,-1,157
7,1.000092e+19,0,14102100,1005,1,e151e245,7e091613,f028772b,ecad2386,7801e8d9,...,1,0,20632,320,50,2374,3,39,-1,23
8,1.000095e+19,1,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,2,15707,320,50,1722,0,35,-1,79
9,1.000126e+19,0,14102100,1002,0,84c7ba46,c4e18dd6,50e219e0,ecad2386,7801e8d9,...,0,0,21689,320,50,2496,3,167,100191,23


In [15]:
pandas_profiling.ProfileReport(dtrain)

Number of variables,24
Number of observations,100000
Total Missing (%),0.0%
Total size in memory,18.3 MiB
Average record size in memory,192.0 B
Numeric,11
Categorical,9
Boolean,1
Date,0
Text (Unique),0
Rejected,3


In [18]:
test.describe()

,id,hour,C1,banner_pos,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
count,4.577464e+06,4.577464e+06,4.577464e+06,4.577464e+06,4.577464e+06,4.577464e+06,4.577464e+06,4.577464e+06,4.577464e+06,4.577464e+06,4.577464e+06,4.577464e+06,4.577464e+06,4.577464e+06
mean,9.223843e+18,1.410311e+07,1.004988e+03,2.110354e-01,1.009555e+00,3.748268e-01,2.171379e+04,3.202237e+02,5.954672e+01,2.469186e+03,1.304855e+00,1.851339e+02,5.347138e+04,9.195548e+01
std,5.325854e+18,5.694117e+00,9.305460e-01,4.724501e-01,4.278750e-01,8.963464e-01,3.765320e+03,2.868091e+01,4.649226e+01,4.756799e+02,1.305181e+00,2.622933e+02,4.994407e+04,8.394011e+01
min,1.282261e+13,1.410310e+07,1.001000e+03,0.000000e+00,0.000000e+00,0.000000e+00,3.750000e+02,1.200000e+02,2.000000e+01,1.120000e+02,0.000000e+00,3.300000e+01,-1.000000e+00,1.300000e+01
25%,4.612842e+18,1.410311e+07,1.005000e+03,0.000000e+00,1.000000e+00,0.000000e+00,2.161100e+04,3.200000e+02,5.000000e+01,2.480000e+03,0.000000e+00,3.500000e+01,-1.000000e+00,2.300000e+01
50%,9.227158e+18,1.410311e+07,1.005000e+03,0.000000e+00,1.000000e+00,0.000000e+00,2.313700e+04,3.200000e+02,5.000000e+01,2.664000e+03,1.000000e+00,3.900000e+01,1.000740e+05,5.100000e+01
75%,1.383712e+19,1.410312e+07,1.005000e+03,0.000000e+00,1.000000e+00,0.000000e+00,2.408500e+04,3.200000e+02,5.000000e+01,2.761000e+03,3.000000e+00,2.950000e+02,1.001110e+05,2.210000e+02
max,1.844674e+19,1.410312e+07,1.012000e+03,7.000000e+00,5.000000e+00,5.000000e+00,2.434900e+04,1.024000e+03,1.024000e+03,2.793000e+03,3.000000e+00,1.831000e+03,1.002480e+05,2.550000e+02


In [17]:
pandas_profiling.ProfileReport(test)

Number of variables,23
Number of observations,4577464
Total Missing (%),0.0%
Total size in memory,803.2 MiB
Average record size in memory,184.0 B
Numeric,13
Categorical,9
Boolean,0
Date,0
Text (Unique),0
Rejected,1


In [2]:
test_read = pd.read_csv('./train.csv',chunksize=500)

In [3]:
for chunk in test_read:
    print(chunk)
    break

                       id  click      hour    C1  banner_pos   site_id  \
0     1000009418151094273      0  14102100  1005           0  1fbe01fe   
1    10000169349117863715      0  14102100  1005           0  1fbe01fe   
2    10000371904215119486      0  14102100  1005           0  1fbe01fe   
3    10000640724480838376      0  14102100  1005           0  1fbe01fe   
4    10000679056417042096      0  14102100  1005           1  fe8cc448   
5    10000720757801103869      0  14102100  1005           0  d6137915   
6    10000724729988544911      0  14102100  1005           0  8fda644b   
7    10000918755742328737      0  14102100  1005           1  e151e245   
8    10000949271186029916      1  14102100  1005           0  1fbe01fe   
9    10001264480619467364      0  14102100  1002           0  84c7ba46   
10   10001868339616595934      0  14102100  1005           1  e151e245   
11   10001966791793526909      0  14102100  1005           0  1fbe01fe   
12   10002028568167339219      0  1410